In [ ]:
#default_exp class_confusion

# 03_class_confusion

> Main code to get `ClassConfusion` running

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from ClassConfusion2.tabular import get_losses, get_names, plot
from ClassConfusion2.vision import get_losses, get_names, plot
import itertools
from fastai2.interpret import ClassificationInterpretation
from google.colab import widgets
from tqdm import tqdm

In [ ]:
#export
class ClassConfusion():
  "Plots the most confused datapoints and statistics for model misses. First is prediction second is actual"
  def __init__(self, learn, dl=None, cut_off=100, is_ordered=False, classlist=[], 
                varlist=None, figsize=(12,12), **kwargs):
    dl = learn.dls[1] if dl is None else dl
    interp = ClassificationInterpretation.from_learner(learn, dl=dl)
    combs = classlist if is_ordered else list(itertools.permutations(classlist, 2))
    figsize = figsize
    cut_off = cut_off
    vocab = interp.vocab
    _, tl_idx = interp.top_losses(len(interp.losses))
    idxs = dl.get_losses(tl_idx, interp.preds, combs)
    mc = interp.most_confused()
    tbnames, boxes, cols, rows, ranges = self._get_names(x=dl, idxs=idxs, mc=mc, varlist=varlist, li=combs)
    tb = widgets.TabBar(tbnames)
    self._create_tabs(tb, tbnames, dl, interp, idxs, combs, boxes, cols, rows, ranges, figsize, cut_off)

  def _create_tabs(self, tb, tbnames, dl, interp, idxs, combs, boxes, cols, rows, ranges, figsize, cut_off):
    "Adds relevant graphs to each tab"
    with tqdm(total=len(tbnames)) as pbar:
      for i, tab in enumerate(tbnames):
        with tb.output_to(i):
          plot(dl, interp, idxs, combs, tab, i, boxes, cols, rows, ranges, figsize, cut_off)
      pbar.update(1)
    
  def _get_names(self, x, idxs, mc, varlist, li):
    return get_names(x, idxs, mc, varlist, li)